In [9]:
import pandas as pd
import json
import ast
import pyarrow.parquet as pq

In [10]:
#Lista para almacenar los diccionarios JSON de cada línea
data_list = []

#Ruta del archivo JSON
file_path = 'Datasets\\JSON\\australian_users_items.json'

#Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
data_users_items = pd.DataFrame(data_list)

In [11]:
#Desanidamos los datos de la columna 'items'
data_user_item1 = data_users_items.explode(['items'])
data_user_item2 = pd.json_normalize(data_user_item1['items']).set_index(data_user_item1['items'].index)
data_user_item3= pd.concat([data_user_item2, data_user_item1], axis=1)
data_users_items = data_user_item3

In [12]:
#Realizamos la eliminacion de las columnas que no necesitamos
df_clean_items = data_users_items.drop('user_url', axis=1)
df_clean_items = df_clean_items.drop('playtime_2weeks', axis=1)
df_clean_items = df_clean_items.drop('items', axis=1)
df_clean_items = df_clean_items.drop('steam_id', axis=1)
df_clean_items = df_clean_items.drop('items_count', axis=1)
df_clean_items = df_clean_items.drop('item_name', axis=1)
data_users_items = df_clean_items

In [13]:
#Realizamos la limpieza de los valores "Nulos" en las columnas que requerimos
df_clean_items = data_users_items.dropna(subset=['item_id', 'playtime_forever'])
data_users_items = df_clean_items

#Definimos Dataframe para juegos donde hayan tenido tiempo jugado
data_users_items = data_users_items[data_users_items['playtime_forever'] != 0]

#Eliminamos valores duplicados y nulos si los hay
data_users_items = data_users_items.drop_duplicates()
data_users_items = data_users_items.dropna()

In [14]:
data_users_items.to_parquet('DatasetsLimpios\\user_items.parquet')